In [1]:
import bs4
import pandas as pd

from urllib.request import urlopen as uReq
from bs4 import BeautifulSoup
from urllib.request import Request
from time import sleep
from requests import get

# First Part - Web Scraping
I Web Scraped an extensive list of Campy Horror Movies from [All Horror.com](https://www.allhorror.com/subgenre/campy?order=release_date&order_dir=asc&sf_paged=). I got the Title, the Year, the genre, and the IMDB Rating.

In [2]:
titles = []
imdb_ratings = []
years = []
genres = []


for i in range(1,54):
    url = 'https://www.allhorror.com/subgenre/campy?order=release_date&order_dir=asc&sf_paged='+ str(i)
    page = get(url, headers={'User-Agent': 'Mozilla/5.0'})
    
    page_soup = BeautifulSoup(page.text, 'html.parser')

    containers = page_soup.findAll('li', {"class": "row"})

    
    for container in containers:
        
        my_str = container.div.a.img['alt']
        
        # Full Movie Exception
        full_movie = my_str.replace('Full Movie', '')
        table = str.maketrans(dict.fromkeys("()"))
        title = full_movie.translate(table)
        
        # Scrape the title
        the_title = title[:-5]
        titles.append(the_title)
        
        # Scrape the year
        word_list = title.split()
        year = word_list[-1]
        years.append(year)
        
        #Scrape the genre
        free = container.findAll('li')
        genre_text = free[0].text
        sep = '|'
        genre = genre_text.split(sep, 1)[0]
        genres.append(genre)
    
        rating_container = container.findAll('li')
               
        # Scrape the IMDB rating
        imdb_rating = rating_container[2].text
        imdb_ratings.append(imdb_rating)
    
    sleep(10)

From here I stored the recorded data in a Data Frame called Movie_Rating.

In [32]:
movie_ratings = pd.DataFrame({'title': titles,
'year': years, 
'imdb': imdb_ratings,
'genre': genres                              
})
print(movie_ratings.info())
movie_ratings.head(20)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 526 entries, 0 to 525
Data columns (total 4 columns):
title    526 non-null object
year     526 non-null object
imdb     526 non-null object
genre    526 non-null object
dtypes: object(4)
memory usage: 16.5+ KB
None


,title,year,imdb,genre
0,Half Human: The Story of the Abominable Snowman,1958,2.9,"Creatures, Thriller"
1,Plan 9 from Outer Space,1959,4.0,"Aliens, Classic, Classic Sci-Fi, Featured Clas..."
2,The Phantom of the Operetta,1960,6.8,Comedy
3,Mr. Sardonicus,1961,6.7,"Comedy, Mystery"
4,House on Bare Mountain,1962,4.1,Comedy
5,House of the Frights,1963,5.0,Comedy
6,The Old Dark House,1963,5.4,Comedy
7,The Incredibly Strange Creatures Who Stopped L...,1964,2.2,"Comedy, Zombies"
8,The Flesh Eaters,1964,6.0,Creatures
9,The Wrestling Women vs. the Aztec Mummy,1964,3.9,Comedy


## Clean Data
There were several issue with the data received. I had to make sure I recorded only the Year.

In [53]:
movie_ratings['year'].unique()

array(['1958', '1959', '1960', '1961', '1962', '1963', '1964', '1965',
       '1966', '1967', '1968', '1969', '1970', '1971', '1972', '1973',
       '1974', '1975', '1976', '1977', '1978', '1979', '1980', '1981',
       '1982', '1983', '1984', '1985', '1986', 'Short', '1987', '1988',
       '1989', '1990', '1990h', '1991', '1992', '1993', '1994', '1995',
       '1996', '1997', '1998', '1999', '2000', '2001', '2002', '2003',
       '2005', '2006', '2007', '2008', '2009', '2010', '2011', '2012',
       '2013', '2014', '2015', '2016', '2017', '2018', '2019'],
      dtype=object)

In [54]:
movie_rating.loc[movie_ratings['year'] == '1990h']

,title,year,imdb,genre
314,Puppet Master II,1990h,5.5,


In [55]:
campy_movies = movie_ratings[movie_ratings.year != 'Short']

In [56]:
campy_movies.head(20)

,title,year,imdb,genre
0,Half Human: The Story of the Abominable Snowman,1958,2.9,"Creatures, Thriller"
1,Plan 9 from Outer Space,1959,4.0,"Aliens, Classic, Classic Sci-Fi, Featured Clas..."
2,The Phantom of the Operetta,1960,6.8,Comedy
3,Mr. Sardonicus,1961,6.7,"Comedy, Mystery"
4,House on Bare Mountain,1962,4.1,Comedy
5,House of the Frights,1963,5.0,Comedy
6,The Old Dark House,1963,5.4,Comedy
7,The Incredibly Strange Creatures Who Stopped L...,1964,2.2,"Comedy, Zombies"
8,The Flesh Eaters,1964,6.0,Creatures
9,The Wrestling Women vs. the Aztec Mummy,1964,3.9,Comedy


Got rid of '1990h'

In [57]:
campy_movies.loc[314, 'year'].replace('h', '')

'1990'

In [58]:
campy_movies.loc[314, 'year'] = campy_movies.loc[314, 'year'].replace('h', '')

In [59]:
campy_movies['year'].unique()

array(['1958', '1959', '1960', '1961', '1962', '1963', '1964', '1965',
       '1966', '1967', '1968', '1969', '1970', '1971', '1972', '1973',
       '1974', '1975', '1976', '1977', '1978', '1979', '1980', '1981',
       '1982', '1983', '1984', '1985', '1986', '1987', '1988', '1989',
       '1990', '1991', '1992', '1993', '1994', '1995', '1996', '1997',
       '1998', '1999', '2000', '2001', '2002', '2003', '2005', '2006',
       '2007', '2008', '2009', '2010', '2011', '2012', '2013', '2014',
       '2015', '2016', '2017', '2018', '2019'], dtype=object)

In [60]:
campy_movies.loc[:, 'year'] = campy_movies['year'].astype(int)

In [61]:
campy_movies.head()

,title,year,imdb,genre
0,Half Human: The Story of the Abominable Snowman,1958,2.9,"Creatures, Thriller"
1,Plan 9 from Outer Space,1959,4.0,"Aliens, Classic, Classic Sci-Fi, Featured Clas..."
2,The Phantom of the Operetta,1960,6.8,Comedy
3,Mr. Sardonicus,1961,6.7,"Comedy, Mystery"
4,House on Bare Mountain,1962,4.1,Comedy


Replaced N/A with 0 in order to better deal with data. Changed IMDB Rating dat type to float, to better manmipulate data. 

In [62]:
campy_movies.loc[:, 'imdb'] = campy_movies.loc[:, 'imdb'].replace('  N/A', '0')

In [63]:
campy_movies.loc[:, 'imdb'] = campy_movies.loc[:, 'imdb'].str.strip()

In [64]:
campy_movies.head()

,title,year,imdb,genre
0,Half Human: The Story of the Abominable Snowman,1958,2.9,"Creatures, Thriller"
1,Plan 9 from Outer Space,1959,4.0,"Aliens, Classic, Classic Sci-Fi, Featured Clas..."
2,The Phantom of the Operetta,1960,6.8,Comedy
3,Mr. Sardonicus,1961,6.7,"Comedy, Mystery"
4,House on Bare Mountain,1962,4.1,Comedy


In [65]:
campy_movies.loc[:, 'imdb'] = campy_movies['imdb'].astype(float)

In [66]:
movie_ratings['imdb'].head(3)

0      2.9
1      4.0
2      6.8
Name: imdb, dtype: object

In [67]:
campy_movies.describe().loc[['min', 'max'], ['imdb']]

,imdb
min,0.0
max,8.1


In [68]:
campy_movies['n_imdb'] = campy_movies['imdb'] * 10
campy_movies.head(3)

,title,year,imdb,genre,n_imdb
0,Half Human: The Story of the Abominable Snowman,1958,2.9,"Creatures, Thriller",29.0
1,Plan 9 from Outer Space,1959,4.0,"Aliens, Classic, Classic Sci-Fi, Featured Clas...",40.0
2,The Phantom of the Operetta,1960,6.8,Comedy,68.0


Saved data stored to Campy Movies.

In [69]:
campy_movies.to_csv('campy_movies.csv')